In [1]:
import os
from bs4 import BeautifulSoup
import requests
import urllib
import re
import csv
import pandas as pd
import nltk
import string
import numpy
import matplotlib.pyplot as plt
%matplotlib inline
from nltk import sent_tokenize, word_tokenize
import pickle
import enchant
from guess_language import guess_language

In [2]:
path='data/pickles/'

In [3]:
file=os.listdir('data/pickles/')

In [4]:
file

['lang8eng.pkl',
 'total_set_0201.pkl',
 'lang8kor.pkl',
 'total_eng_set_0201.pkl',
 'naver_q.pkl',
 'total_kor_set_0202.pkl',
 'model_tot',
 'model_eng',
 'lang8_kor_1.pkl',
 'eng_large_pd.pkl',
 'model_kor',
 'train_set_0201.pkl',
 'eng_large_new_pd',
 'kor_large_pd.pkl',
 'train_set_0202.pkl',
 'nltk_eng.pkl',
 'uvecmodel',
 'univ_kor.pkl',
 'total_set_0202.pkl',
 'test_set_0202.pkl',
 'lang8_kor_new.pkl',
 'total_set.pkl',
 'eng_large_new_pd.pkl',
 'naver_pd.pkl',
 'total_kor_set_0201.pkl',
 'total_eng_set_0202.pkl',
 'test_set_0201.pkl',
 'cv_set_0202.pkl',
 'naver_r.pkl',
 'cv_set_0201.pkl',
 'lang8_eng_3']

In [91]:
eng_list=pickle.load(open('wordvec/nltk_eng.pkl', 'rb'))

kor_list=pickle.load(open('wordvec/univ_kor.pkl', 'rb'))

In [92]:
naver_q=pickle.load(open(path+'naver_q.pkl', 'rb'))
naver_r=pickle.load(open(path+'naver_r.pkl', 'rb'))
kor_lang8=pickle.load(open(path+'lang8_kor_1.pkl','rb'))
eng_lang8=pickle.load(open(path+'lang8_eng_3','rb'))

In [93]:
Naver=naver_q+naver_r

In [94]:
len(Naver)

32153

In [95]:
len(kor_list)

238644

# Trying to clean up the duplicates and the none english sentences

In [96]:
kor_lang8 = list(filter(None, kor_lang8))
eng_lang8 = list(filter(None, eng_lang8))

In [97]:
def Lang8_clean(somelist):
    somelist = list(filter(None, somelist))
    eng_only_list=[]
    for item in somelist:
        score=0
        sentence=''
        for word in item:
            sentence=sentence+' '+word
            if guess_language(word)=='en':
                score=score+1        
        if score/len(item)>0.6:
            eng_only_list.append(sentence)
    return eng_only_list
    

In [98]:
print (len(eng_lang8))

86261


In [99]:
print (len(eng_lang8_clean))

58112


In [100]:
eng_lang8_clean=Lang8_clean(eng_lang8)

In [63]:
df_l8e = pd.DataFrame({'eng_vec' : eng_lang8_clean })

In [65]:
df_l8e=df_l8e.drop_duplicates()

In [66]:
len(df_l8e)

35631

In [102]:
kor_list_new=[]
for item in kor_lang8:
    score=0
    sentence=''
    for word in item:
        sentence=sentence+' '+word
        if guess_language(word)=='en':
            score=score+1        
    if score/len(item)>0.6:
        kor_list_new.append(sentence)
#         sentence=sentence+' '+word
#     if guess_language(sentence)!='en':
#         if guess_language(sentence) == 'UNKNOWN':
#             print (sentence)

In [39]:
df = pd.DataFrame({'kor_vec' : kor_list_new })

In [40]:
df=df.drop_duplicates()

In [44]:
dnaver = pd.DataFrame({'kor_vec' : Naver })

In [47]:
dlang8=pd.DataFrame({'kor_vec' : })

,kor_vec
0,ese day i am so tired because i woke up a eve...
1,i have a stomachach since 2 weeks ago
2,i thought everything is gon na be but i very ...
3,doctor said you have a convulsion
4,you have to avoid coffee wheat flour and milk


Dataframes to be made:

    deng_large
    eng_list : from original pickle data list saved as eng_list
    (eng_list=pickle.load(open('wordvec/nltk_eng.pkl', 'rb')))
    
    dkor_large
    kor_list=pickle.load(open('wordvec/univ_kor.pkl', 'rb'))
    (kor_list : from original pickle data list saved as kor_list)
    
    dnaver
    naver_list : broken down to Q&R originally saved as Naver (Only Kor sample from here)
    
    dlang8kor
    Lang-8 Korean list : kor_list_new with some duplicates make sure to do drop_duplicates
    dlang8eng
    Lang-8 Ennglish list:  eng_lang8_clean with some duplicates
    
    

In [67]:
deng_large = pd.DataFrame({'sent' : eng_list, 'label': 0})
dkor_large = pd.DataFrame({'sent' : kor_list, 'label': 1})

In [68]:
dnaver = pd.DataFrame({'sent' : Naver, 'label': 1 })

In [115]:
dlang8eng=pd.DataFrame({'sent' : eng_lang8_clean,'label':0})
dlang8kor=pd.DataFrame({'sent' : kor_list_new, 'label':1 })

In [116]:
dlang8eng['sent']=dlang8eng['sent'].drop_duplicates()
dlang8kor['sent']=dlang8kor['sent'].drop_duplicates()

In [120]:
dlang8eng.tail()

,label,sent
55182,0,what a sad sight
55183,0,i am currently listening to this talented lad...
55184,0,i simply love jazz
55185,0,do you like it too
55186,0,i like the song below very much how


In [121]:
dlang8eng=dlang8eng.dropna(subset=['sent'])
dlang8kor=dlang8kor.dropna(subset=['sent'])

In [122]:
dlang8eng['sent'] = dlang8eng['sent'].apply(lambda x: word_tokenize(str(x)))

In [123]:
dlang8kor['sent'] = dlang8kor['sent'].apply(lambda x: word_tokenize(str(x)))

In [124]:
dlang8eng.tail()

,label,sent
55182,0,"[what, a, sad, sight]"
55183,0,"[i, am, currently, listening, to, this, talent..."
55184,0,"[i, simply, love, jazz]"
55185,0,"[do, you, like, it, too]"
55186,0,"[i, like, the, song, below, very, much, how]"


In [125]:
dlang8kor.head()

,label,sent
0,1,"[ese, day, i, am, so, tired, because, i, woke,..."
1,1,"[i, have, a, stomachach, since, 2, weeks, ago]"
2,1,"[i, thought, everything, is, gon, na, be, but,..."
3,1,"[doctor, said, you, have, a, convulsion]"
4,1,"[you, have, to, avoid, coffee, wheat, flour, a..."


In [ ]:
deng_large.to_pickle(path+'eng_large_pd.pkl')
dkor_large.to_pickle(path+'kor_large_pd.pkl')
dnaver.to_pickle(path+'naver_pd.pkl')

In [126]:
dlang8eng.to_pickle(path+'lang8eng.pkl')
dlang8kor.to_pickle(path+'lang8kor.pkl')

In [85]:
dlang8kor.head()

0    [ese, day, i, am, so, tired, because, i, woke,...
1       [i, have, a, stomachach, since, 2, weeks, ago]
2    [i, thought, everything, is, gon, na, be, but,...
3             [doctor, said, you, have, a, convulsion]
4    [you, have, to, avoid, coffee, wheat, flour, a...
Name: sent, dtype: object